# Imports

In [5]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import glob
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import  RepeatedStratifiedKFold
import kagglehub
from ResNet34 import ResNetTrainer
from sklearn.model_selection import RandomizedSearchCV
from DatasetClasses import CapsuleDataset
from wrappers import CNNVAEResNetEstimator,CNNVAEWrapper, CNNGANWrapper, CNNGANResNetEstimator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Parametry modelu
IMG_SIZE = 128
CHANNELS = 3
LATENT_DIM = 64
HIDDEN_DIM = 512
BATCH_SIZE = 16
EPOCHS = 100

# Konfiguracja
result_dir = 'results/'
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

result_dir = 'results/GAN'
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

result_dir = 'results/CNNGAN'
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

result_dir = 'results/VAE'
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

result_dir = 'results/CNNVAE'
if not os.path.exists(result_dir):
    os.makedirs(result_dir)


device = (
    torch.device("mps") if torch.backends.mps.is_available()
    else torch.device("cuda") if torch.cuda.is_available()
    else torch.device("cpu")
)

if torch.backends.mps.is_available():
    torch.mps.empty_cache()

print(f"Training device: {device}")

Training device: cpu


In [7]:
rskf = RepeatedStratifiedKFold(
    n_splits=5,
    n_repeats=2,
    random_state=42
)

# Dataset preparing

In [4]:
# your augmentations + normalization
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomAffine(degrees=15, translate=(0.05,0.05), scale=(1.1,1.15), fill=255),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
])   

path = kagglehub.dataset_download("tladilebohang/capsule-defects")
# download or mount your Kaggle data however you like; suppose:
# path = ".../capsule-defects"
pos_folder = os.path.join(path, "capsule/positive")
neg_folder = os.path.join(path, "capsule/negative")
pos_len=len(glob.glob(os.path.join(pos_folder, "*")))
print(pos_len)
neg_len=len(glob.glob(os.path.join(neg_folder, "*")))
print(neg_len)

219
109


# Eksperyment na datasecie bez oversamplingu

In [30]:
IMG_SIZE   = 128
BATCH_SIZE = 16
CHANNELS   = 3
EPOCHS = 25

dataset = CapsuleDataset(pos_dir=pos_folder, neg_dirs=neg_folder, transform=transform)
print(dataset.__len__())

labels = np.array([dataset[i][1] for i in range(len(dataset))])

results = []

for fold_idx, (train_idx, test_idx) in enumerate(rskf.split(X=np.zeros(len(labels)), y=labels), start=1):
    print(f"===== Fold {fold_idx} =====")

    # Subset + DataLoader
    train_ds = Subset(dataset, train_idx)
    test_ds  = Subset(dataset, test_idx)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=True)
    trainer = ResNetTrainer()
    
    trainer.train(
        train_loader,
        num_epochs=EPOCHS,
    )

    f2, bal_acc, recall, specificity = trainer.validate(test_loader)
    print(f"Fold {fold_idx} Test Accuracy: {bal_acc:.4f}")

    results.append({
        'fold': fold_idx,
        'f2_score': f2,
        'balanced_accuracy': bal_acc,
        'recall': recall,
        'specificity': specificity
    })

#Zapis wyników z każdego folda
results_df = pd.DataFrame(results)
results_df.to_csv('without_oversampling_cross_validation_results.csv', index=False)


328
===== Fold 1 =====
Epoch 1/25 | Train Loss: 0.7442 Acc: 0.6336
Epoch 2/25 | Train Loss: 0.6194 Acc: 0.7023
Epoch 3/25 | Train Loss: 0.6195 Acc: 0.6908
Epoch 4/25 | Train Loss: 0.5723 Acc: 0.7176
Epoch 5/25 | Train Loss: 0.4970 Acc: 0.7786
Epoch 6/25 | Train Loss: 0.6026 Acc: 0.7290
Epoch 7/25 | Train Loss: 0.5569 Acc: 0.7366
Epoch 8/25 | Train Loss: 0.4960 Acc: 0.7672
Epoch 9/25 | Train Loss: 0.4983 Acc: 0.7557
Epoch 10/25 | Train Loss: 0.4775 Acc: 0.7672


KeyboardInterrupt: 

# CNNVAE: Ekperyment 

In [124]:

IMG_SIZE   = 128
BATCH_SIZE = 16
CHANNELS   = 3
CLASSIFIER_EPOCHS = 25
OVERSAMPLER_EPOCHS = 200

dataset = CapsuleDataset(pos_dir=pos_folder, neg_dirs=neg_folder, transform=transform)
print(dataset.__len__())

labels = np.array([dataset[i][1] for i in range(len(dataset))])

results = []

for fold_idx, (train_idx, test_idx) in enumerate(rskf.split(X=np.zeros(len(labels)), y=labels), start=1):
    print(f"===== Fold {fold_idx} =====")

    
    CnnVae = CNNVAEWrapper(dataset,device,BATCH_SIZE,OVERSAMPLER_EPOCHS)
    
    CnnVae.fit(train_idx)


    estimator = CNNVAEResNetEstimator(
        dataset=dataset,
        vae_model=CnnVae.vae_model,
        device=device,
        batch_size=BATCH_SIZE,
        classifier_epochs=CLASSIFIER_EPOCHS
    )

    param_grid = {
        'mu_multiplier': [0.8, 1.2],
        'logvar_multiplier': [0.5, 1.5],
        'multiplier_generated_samples': [1/2,1, 2]
    }


    random_search = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=param_grid,
        n_iter=8, 
        cv=None,
        verbose=2,
        n_jobs=1,
        random_state=42
    )
    
    random_search.fit(train_idx)

    best_estimator = random_search.best_estimator_
    best_params = random_search.best_params_

    test_ds = Subset(dataset, test_idx)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
    f2, bal_acc, recall, specificity = best_estimator.trainer.validate(test_loader)
    print(f"Fold {fold_idx} Test Accuracy: {bal_acc:.4f}")

    results.append({
        'fold': fold_idx,
        'f2_score': f2,
        'balanced_accuracy': bal_acc,
        'recall': recall,
        'specificity': specificity,
        **best_params  
    })




#Zapis wyników z każdego folda
results_df = pd.DataFrame(results)
results_df.to_csv('CNNVAE_cross_validation_results.csv', index=False)
    

328
===== Fold 1 =====
Train Epoch: 1 | Loss: 2304.4632
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Epoch 1/1 | Train Loss: 0.5439 Acc: 0.6841
[CV] END logvar_multiplier=0.5, mu_multiplier=1.2, multiplier_generated_samples=2; total time=  54.1s
Epoch 1/1 | Train Loss: 0.5428 Acc: 0.6736
[CV] END logvar_multiplier=0.5, mu_multiplier=1.2, multiplier_generated_samples=2; total time=  54.5s
Epoch 1/1 | Train Loss: 0.4985 Acc: 0.7526
[CV] END logvar_multiplier=0.5, mu_multiplier=1.2, multiplier_generated_samples=2; total time=  56.1s
Epoch 1/1 | Train Loss: 0.6003 Acc: 0.7580
[CV] END logvar_multiplier=0.5, mu_multiplier=1.2, multiplier_generated_samples=2; total time=  48.9s
Epoch 1/1 | Train Loss: 0.4814 Acc: 0.8107
[CV] END logvar_multiplier=0.5, mu_multiplier=1.2, multiplier_generated_samples=2; total time=  43.4s
Epoch 1/1 | Train Loss: 0.7441 Acc: 0.5652
[CV] END logvar_multiplier=1.0, mu_multiplier=1.0, multiplier_generated_samples=0.5; total time=  40.4s
Epoch 1/1 | 

KeyboardInterrupt: 

# CNNGAN: Eksperyment

In [6]:

IMG_SIZE   = 128
BATCH_SIZE = 16
CHANNELS   = 3
CLASSIFIER_EPOCHS = 15
OVERSAMPLER_EPOCHS = 250

dataset = CapsuleDataset(pos_dir=pos_folder, neg_dirs=neg_folder, transform=transform)
print(dataset.__len__())

labels = np.array([dataset[i][1] for i in range(len(dataset))])

results = []

for fold_idx, (train_idx, test_idx) in enumerate(rskf.split(X=np.zeros(len(labels)), y=labels), start=1):
    print(f"===== Fold {fold_idx} =====")


    CnnGan = CNNGANWrapper(dataset,device,BATCH_SIZE,OVERSAMPLER_EPOCHS)

    CnnGan.fit(train_idx)


    estimator = CNNGANResNetEstimator(
        dataset=dataset,
        gan_model=CnnGan.gan_model,
        device=device,
        batch_size=BATCH_SIZE,
        classifier_epochs=CLASSIFIER_EPOCHS
    )

    param_grid = {
        'scale_factor': [0.5, 1, 1.5],  # Czynnik skalujący szum w GAN
        'multiplier_generated_samples': [1/2,1, 2]
    }


    random_search = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=param_grid,
        n_iter=6,
        cv=None,
        verbose=2,
        n_jobs=1,
        random_state=42
    )

    random_search.fit(train_idx)

    best_estimator = random_search.best_estimator_
    best_params = random_search.best_params_

    test_ds = Subset(dataset, test_idx)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
    f2, bal_acc, recall, specificity = best_estimator.trainer.validate(test_loader)
    print(f"Fold {fold_idx} Test Accuracy: {bal_acc:.4f}")

    results.append({
        'fold': fold_idx,
        'f2_score': f2,
        'balanced_accuracy': bal_acc,
        'recall': recall,
        'specificity': specificity,
        **best_params
    })




#Zapis wyników z każdego folda
results_df = pd.DataFrame(results)
results_df.to_csv('CNNGAN_cross_validation_results.csv', index=False)

328
===== Fold 1 =====
Epoch 1: Generator Loss = 10.0291, Discriminator Loss = 8.6379
Epoch 2: Generator Loss = 10.9063, Discriminator Loss = 8.2331
Epoch 3: Generator Loss = 12.0662, Discriminator Loss = 6.6395
Epoch 4: Generator Loss = 18.0754, Discriminator Loss = 5.1761
Epoch 5: Generator Loss = 26.0159, Discriminator Loss = 4.0308
Epoch 6: Generator Loss = 29.1323, Discriminator Loss = 5.6949
Epoch 7: Generator Loss = 24.4812, Discriminator Loss = 4.9665
Epoch 8: Generator Loss = 25.7317, Discriminator Loss = 5.7915
Epoch 9: Generator Loss = 28.1827, Discriminator Loss = 6.6236
Epoch 10: Generator Loss = 21.3980, Discriminator Loss = 7.9283
Epoch 11: Generator Loss = 21.2976, Discriminator Loss = 7.0885
Epoch 12: Generator Loss = 23.1601, Discriminator Loss = 8.2236
Epoch 13: Generator Loss = 21.8036, Discriminator Loss = 8.0966
Epoch 14: Generator Loss = 17.0830, Discriminator Loss = 8.8234
Epoch 15: Generator Loss = 18.3067, Discriminator Loss = 7.6943
Epoch 16: Generator Loss =

# CNNGAN: Generowanie jedynie syntetycznego zbioru

In [8]:

IMG_SIZE   = 128
BATCH_SIZE = 16
CHANNELS   = 3
CLASSIFIER_EPOCHS = 20
OVERSAMPLER_EPOCHS = 200

dataset = CapsuleDataset(pos_dir=pos_folder, neg_dirs=neg_folder, transform=transform)
print(dataset.__len__())

labels = np.array([dataset[i][1] for i in range(len(dataset))])

results = []

for fold_idx, (train_idx, test_idx) in enumerate(rskf.split(X=np.zeros(len(labels)), y=labels), start=1):
    print(f"===== Fold {fold_idx} =====")


    CnnGan = CNNGANWrapper(dataset,device,BATCH_SIZE,OVERSAMPLER_EPOCHS)

    CnnGan.fit(train_idx)


    estimator = CNNGANResNetEstimator(
        dataset=dataset,
        gan_model=CnnGan.gan_model,
        device=device,
        batch_size=BATCH_SIZE,
        classifier_epochs=CLASSIFIER_EPOCHS,
        multiplier_generated_samples='synthetic'
    )

    param_grid = {
        'scale_factor': [0.5, 1, 1.5]  # Czynnik skalujący szum w GAN
    }


    random_search = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=param_grid,
        n_iter=2,
        cv=None,
        verbose=2,
        n_jobs=1,
        random_state=42
    )

    random_search.fit(train_idx)

    best_estimator = random_search.best_estimator_
    best_params = random_search.best_params_

    test_ds = Subset(dataset, test_idx)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
    f2, bal_acc, recall, specificity = best_estimator.trainer.validate(test_loader)
    print(f"Fold {fold_idx} Test Accuracy: {bal_acc:.4f}")

    results.append({
        'fold': fold_idx,
        'f2_score': f2,
        'balanced_accuracy': bal_acc,
        'recall': recall,
        'specificity': specificity,
        **best_params
    })




#Zapis wyników z każdego folda
results_df = pd.DataFrame(results)
results_df.to_csv('CNNGAN_synthetic_cross_validation_results.csv', index=False)

328
===== Fold 1 =====
Epoch 1: Generator Loss = 10.4548, Discriminator Loss = 9.1613
Epoch 2: Generator Loss = 13.1622, Discriminator Loss = 5.4860
Epoch 3: Generator Loss = 22.3018, Discriminator Loss = 3.5493
Epoch 4: Generator Loss = 32.5408, Discriminator Loss = 3.3099
Epoch 5: Generator Loss = 35.1577, Discriminator Loss = 3.9735
Epoch 6: Generator Loss = 34.9560, Discriminator Loss = 5.2573
Epoch 7: Generator Loss = 29.1434, Discriminator Loss = 4.0361
Epoch 8: Generator Loss = 30.0363, Discriminator Loss = 6.0887
Epoch 9: Generator Loss = 27.9666, Discriminator Loss = 5.1654
Epoch 10: Generator Loss = 29.7431, Discriminator Loss = 6.3831
Epoch 11: Generator Loss = 24.3034, Discriminator Loss = 7.0514
Epoch 12: Generator Loss = 22.2792, Discriminator Loss = 8.6292
Epoch 13: Generator Loss = 19.5683, Discriminator Loss = 6.9064
Epoch 14: Generator Loss = 23.0947, Discriminator Loss = 6.2279
Epoch 15: Generator Loss = 21.7222, Discriminator Loss = 6.3125
Epoch 16: Generator Loss =

# CNNVAE: Generowanie jedynie syntetycznego zbioru

In [9]:

IMG_SIZE   = 128
BATCH_SIZE = 16
CHANNELS   = 3
CLASSIFIER_EPOCHS = 20
OVERSAMPLER_EPOCHS = 200

dataset = CapsuleDataset(pos_dir=pos_folder, neg_dirs=neg_folder, transform=transform)
print(dataset.__len__())

labels = np.array([dataset[i][1] for i in range(len(dataset))])

results = []

for fold_idx, (train_idx, test_idx) in enumerate(rskf.split(X=np.zeros(len(labels)), y=labels), start=1):
    print(f"===== Fold {fold_idx} =====")


    CnnVae = CNNVAEWrapper(dataset,device,BATCH_SIZE,OVERSAMPLER_EPOCHS)

    CnnVae.fit(train_idx)


    estimator = CNNVAEResNetEstimator(
        dataset=dataset,
        vae_model=CnnVae.vae_model,
        device=device,
        batch_size=BATCH_SIZE,
        classifier_epochs=CLASSIFIER_EPOCHS,
        multiplier_generated_samples='synthetic'
    )

    param_grid = {
        'mu_multiplier': [0.8, 1.2],
        'logvar_multiplier': [0.5, 1.5]
    }


    random_search = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=param_grid,
        n_iter=3,
        cv=None,
        verbose=2,
        n_jobs=1,
        random_state=42
    )

    random_search.fit(train_idx)

    best_estimator = random_search.best_estimator_
    best_params = random_search.best_params_

    test_ds = Subset(dataset, test_idx)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)
    f2, bal_acc, recall, specificity = best_estimator.trainer.validate(test_loader)
    print(f"Fold {fold_idx} Test Accuracy: {bal_acc:.4f}")

    results.append({
        'fold': fold_idx,
        'f2_score': f2,
        'balanced_accuracy': bal_acc,
        'recall': recall,
        'specificity': specificity,
        **best_params
    })




#Zapis wyników z każdego folda
results_df = pd.DataFrame(results)
results_df.to_csv('CNNVAE_synthetic_cross_validation_results.csv', index=False)

328
===== Fold 1 =====
Train Epoch: 1 | Loss: 2286.4898
Train Epoch: 2 | Loss: 2069.6775
Train Epoch: 3 | Loss: 2025.7096
Train Epoch: 4 | Loss: 2057.5924
Train Epoch: 5 | Loss: 1985.3303
Train Epoch: 6 | Loss: 1909.5665
Train Epoch: 7 | Loss: 1942.2949
Train Epoch: 8 | Loss: 1957.7975
Train Epoch: 9 | Loss: 1889.5994
Train Epoch: 10 | Loss: 1872.8415
Train Epoch: 11 | Loss: 1786.6735
Train Epoch: 12 | Loss: 1746.9870
Train Epoch: 13 | Loss: 1819.9660
Train Epoch: 14 | Loss: 1735.7154
Train Epoch: 15 | Loss: 1762.4043
Train Epoch: 16 | Loss: 1812.3532
Train Epoch: 17 | Loss: 1791.9655
Train Epoch: 18 | Loss: 1740.2554
Train Epoch: 19 | Loss: 1798.1621
Train Epoch: 20 | Loss: 1712.5966
Train Epoch: 21 | Loss: 1770.6167
Train Epoch: 22 | Loss: 1724.4459
Train Epoch: 23 | Loss: 1729.1468
Train Epoch: 24 | Loss: 1780.5416
Train Epoch: 25 | Loss: 1753.7760
Train Epoch: 26 | Loss: 1791.9717
Train Epoch: 27 | Loss: 1767.8451
Train Epoch: 28 | Loss: 1773.9280
Train Epoch: 29 | Loss: 1747.9425


KeyboardInterrupt: 